# 🛡️ TaxGuard AI - Comprehensive Testing Notebook

This notebook allows you to test all components of the TaxGuard AI system:

1. **Multiple Income Sources** - Spouse, multiple jobs, 1099s, rental income
2. **YTD Withholding Tracking** - Accurate refund/owed projections
3. **Tax Calculator** - Core calculation engine
4. **What-If Simulations** - Impact of changes
5. **Basic Recommendations** - 401(k), HSA, IRA
6. **Advanced Strategies** - Business formation, Section 179, real estate, etc.
7. **PII Redaction** - Privacy air gap testing

---

## 📦 Setup & Imports

In [ ]:
import sys
sys.path.insert(0, '.')

from datetime import date, datetime
from pprint import pprint

# Core modules
from tax_constants import (
    FilingStatus, TAX_BRACKETS_2025, STANDARD_DEDUCTION_2025,
    CONTRIBUTION_LIMITS_2025, PAY_PERIODS_PER_YEAR,
    calculate_federal_tax, get_marginal_rate, get_effective_rate,
    get_tax_bracket_info, get_all_constants_for_llm,
)

from models import (
    UserFinancialProfile, PayFrequency, TaxResult,
    IncomeSource as ModelIncomeSource,
)

# Enhanced models for multiple income sources
from enhanced_models import (
    EnhancedUserProfile, IncomeSource, IncomeSourceType, 
    SpouseIncome, InvestmentIncome, PayFrequency as EnhancedPayFrequency,
)

from pii_redaction import PIIRedactor, redact_sensitive_data

from tax_simulator import (
    TaxCalculator, TaxSimulator, RecommendationEngine, IncomeProjector,
)

# Advanced strategies
from advanced_strategies import (
    get_all_strategies, StrategyCategory, StrategyComplexity,
)

print("✅ All modules imported successfully!")

---
## 1️⃣ Multiple Income Sources

Support for married couples with two incomes, side gigs, rental properties, etc.

In [ ]:
# Create an enhanced profile with multiple income sources
profile = EnhancedUserProfile(
    filing_status=FilingStatus.MARRIED_FILING_JOINTLY,
    age=42,
    num_children_under_17=2
)

# Add taxpayer's primary job
taxpayer_job = IncomeSource(
    source_type=IncomeSourceType.W2_PRIMARY,
    name="[EMPLOYER_1]",  # Redacted
    pay_frequency=EnhancedPayFrequency.BIWEEKLY,
    current_pay_period=22,
    ytd_gross=95000,
    ytd_federal_withheld=12500,
    ytd_state_withheld=4750,
    ytd_401k=15000,
    ytd_hsa=2500,
)
profile.add_income_source(taxpayer_job)

# Add spouse's job
spouse_job = IncomeSource(
    source_type=IncomeSourceType.W2_SPOUSE,
    name="[EMPLOYER_2]",
    pay_frequency=EnhancedPayFrequency.SEMIMONTHLY,
    current_pay_period=20,
    ytd_gross=75000,
    ytd_federal_withheld=9000,
    ytd_state_withheld=3500,
    ytd_401k=12000,
)

# Initialize spouse and add their income
profile.spouse = SpouseIncome(age=40)
profile.spouse.sources.append(spouse_job)

# Add a side gig (1099 work)
side_gig = IncomeSource(
    source_type=IncomeSourceType.FORM_1099_NEC,
    name="Freelance Consulting",
    is_self_employment=True,
    estimated_annual_amount=15000,
    estimated_tax_payments=3000,  # Q1-Q3 payments
)
profile.add_income_source(side_gig)

print("📊 HOUSEHOLD INCOME SUMMARY")
print("=" * 50)
print(f"Income Sources: {profile.total_income_sources}")
print(f"\nTaxpayer W-2 Income:")
print(f"  YTD Gross: ${profile.total_ytd_w2_income:,.2f}")
print(f"  Projected Annual: ${profile.projected_annual_w2_income:,.2f}")
print(f"\nSpouse W-2 Income:")
print(f"  YTD Gross: ${profile.spouse.total_ytd_income:,.2f}")
print(f"  Projected Annual: ${profile.spouse.projected_annual_income:,.2f}")
print(f"\nSelf-Employment: ${profile.total_self_employment_income:,.2f}")
print(f"\nTOTAL Household Income: ${profile.total_projected_income:,.2f}")

In [ ]:
# YTD Withholding Tracking - CRITICAL for accurate refund/owed
print("💰 YTD WITHHOLDING & PAYMENTS")
print("=" * 50)
print(f"Taxpayer Federal Withheld: ${profile.total_ytd_federal_withheld:,.2f}")
print(f"Spouse Federal Withheld: ${profile.spouse.total_federal_withheld:,.2f}")
print(f"Estimated Tax Payments: ${profile.total_estimated_payments:,.2f}")
print(f"\nTOTAL Projected Payments: ${profile.total_projected_payments:,.2f}")

---
## 2️⃣ Tax Calculation with Multiple Sources

Calculate taxes using all income sources with proper withholding projection.

In [ ]:
# Convert enhanced profile to standard profile for calculation
std_profile = UserFinancialProfile(
    filing_status=profile.filing_status,
    age=profile.age,
    spouse_age=profile.spouse.age if profile.spouse else None,
    num_children_under_17=profile.num_children_under_17,
    
    # Combined income
    ytd_income=profile.total_ytd_w2_income + (profile.spouse.total_ytd_income if profile.spouse else 0),
    pay_frequency=PayFrequency.BIWEEKLY,
    current_pay_period=22,
    
    # Combined withholding
    ytd_federal_withheld=profile.total_ytd_federal_withheld + 
                         (profile.spouse.total_federal_withheld if profile.spouse else 0),
    estimated_payments_made=profile.total_estimated_payments,
    
    # Self-employment
    self_employment_income=profile.total_self_employment_income,
    
    # Retirement contributions
    ytd_401k_traditional=profile.ytd_401k_traditional + 
                         (profile.spouse.total_401k if profile.spouse else 0),
    ytd_hsa=profile.ytd_hsa,
    hsa_coverage_type="family",
    has_workplace_retirement_plan=True,
)

# Calculate taxes
calculator = TaxCalculator()
result = calculator.calculate_tax(std_profile)

print("📊 TAX CALCULATION RESULT")
print("=" * 60)
print(f"Gross Income:              ${result.gross_income:>15,.2f}")
print(f"Adjustments:               ${result.adjustments:>15,.2f}")
print(f"Adjusted Gross Income:     ${result.adjusted_gross_income:>15,.2f}")
print(f"{result.deduction_type.title()} Deduction:        ${result.deduction_amount:>15,.2f}")
print(f"Taxable Income:            ${result.taxable_income:>15,.2f}")
print("=" * 60)
print(f"Federal Tax:               ${result.federal_tax:>15,.2f}")
print(f"Self-Employment Tax:       ${result.self_employment_tax:>15,.2f}")
print(f"Child Tax Credit:          ${result.child_tax_credit:>15,.2f}")
print(f"Total Tax Liability:       ${result.total_tax_liability:>15,.2f}")
print("=" * 60)
print(f"Projected Withholding:     ${result.total_payments_and_withholding:>15,.2f}")
print("=" * 60)
if result.refund_or_owed >= 0:
    print(f"✅ PROJECTED REFUND:        ${result.refund_or_owed:>15,.2f}")
else:
    print(f"⚠️ PROJECTED OWED:          ${abs(result.refund_or_owed):>15,.2f}")

---
## 3️⃣ What-If Simulations

In [ ]:
simulator = TaxSimulator(std_profile)

# Scenario 1: Max out remaining 401(k) room
sim_401k = simulator.find_optimal_401k()
print(f"🎯 Max 401(k): Save ${abs(sim_401k.tax_difference):,.2f}")
print(f"   Room remaining: ${std_profile.remaining_401k_room:,.2f}")

# Scenario 2: Max out HSA
sim_hsa = simulator.find_optimal_hsa()
print(f"🎯 Max HSA: Save ${abs(sim_hsa.tax_difference):,.2f}")
print(f"   Room remaining: ${std_profile.remaining_hsa_room:,.2f}")

# Custom scenario: Add $10k to 401k + max HSA
custom = simulator.run_simulation({
    'extra_401k_traditional': 10000,
    'extra_hsa': std_profile.remaining_hsa_room
}, "Add $10k 401k + Max HSA")
print(f"🎯 Custom Scenario: Save ${abs(custom.tax_difference):,.2f}")

---
## 4️⃣ Basic Recommendations

In [ ]:
engine = RecommendationEngine()
report = engine.generate_recommendations(std_profile)

print("💡 BASIC RECOMMENDATIONS")
print("=" * 60)
print(f"Days until year-end: {report.days_until_year_end}")
print(f"Max potential savings: ${report.max_potential_savings:,.2f}")
print()

for rec in report.basic_recommendations[:5]:
    emoji = {"high": "🔴", "medium": "🟡", "low": "🟢"}.get(rec.priority.value, "⚪")
    print(f"{emoji} {rec.title}")
    print(f"   Potential Savings: ${rec.potential_tax_savings:,.2f}")
    print(f"   Action: {rec.action_required[:80]}...")
    print()

---
## 5️⃣ Advanced Life-Changing Strategies

These are the "uncommon" strategies that can significantly reduce taxes through lifestyle changes.

In [ ]:
# Get all advanced strategies
all_strategies = get_all_strategies()

print(f"📚 TOTAL ADVANCED STRATEGIES: {len(all_strategies)}")
print()

# Group by category
from collections import defaultdict
by_category = defaultdict(list)
for s in all_strategies:
    by_category[s.category].append(s)

for cat, strats in sorted(by_category.items(), key=lambda x: len(x[1]), reverse=True):
    print(f"📂 {cat.value.replace('_', ' ').title()}: {len(strats)} strategies")

In [ ]:
# Show top strategies by category
print("🚀 TOP ADVANCED STRATEGIES BY CATEGORY")
print("=" * 70)

for s in all_strategies[:15]:
    badge = {"moderate":"🟢", "advanced":"🟡", "expert":"🔴"}.get(s.complexity.value, "")
    print(f"\n{badge} {s.name}")
    print(f"   Category: {s.category.value.replace('_', ' ').title()}")
    print(f"   Min Income: ${s.min_income:,.0f}")
    print(f"   Potential: {s.potential_savings}")
    if s.description:
        print(f"   {s.description[:100]}...")

In [ ]:
# Deep dive: Business Formation Strategies
print("🔍 BUSINESS FORMATION STRATEGIES")
print("=" * 70)

biz_strats = [s for s in all_strategies if s.category == StrategyCategory.BUSINESS_FORMATION]

for s in biz_strats[:3]:
    print(f"\n🏢 {s.name}")
    print(f"   Complexity: {s.complexity.value}")
    print(f"   Potential: {s.potential_savings}")
    if s.description:
        print(f"   {s.description[:200]}...")
    if s.steps:
        print("   Steps:")
        for i, step in enumerate(s.steps[:3], 1):
            print(f"      {i}. {step}")

In [ ]:
# Deep dive: Real Estate Strategies
print("🔍 REAL ESTATE TAX STRATEGIES")
print("=" * 70)

re_strats = [s for s in all_strategies if s.category == StrategyCategory.REAL_ESTATE]

for s in re_strats[:3]:
    print(f"\n🏠 {s.name}")
    print(f"   Complexity: {s.complexity.value}")
    print(f"   Potential: {s.potential_savings}")
    if s.description:
        print(f"   {s.description[:200]}...")

---
## 6️⃣ PII Redaction Demo

The "air gap" that ensures no personal information reaches AI services.

In [ ]:
# Sample paystub with PII
sample_paystub = """
ACME Corporation
Employee: John A. Smith
SSN: 123-45-6789
Employee ID: E12345

Pay Period: 10/01/2025 - 10/15/2025
Pay Date: 10/20/2025

Address: 123 Main Street, Apt 4B, Anytown, CA 90210
Email: john.smith@acme.com
Phone: (555) 123-4567

Gross Pay: $4,250.00
Federal Tax: $425.00
State Tax: $212.50
Social Security: $263.50
Medicare: $61.63
401(k): $425.00

YTD Gross: $85,000.00
YTD Federal: $8,500.00

Employer EIN: 12-3456789
"""

redactor = PIIRedactor(use_ner=False)
result = redactor.redact_sensitive_data(sample_paystub)

print("🔒 PII REDACTION RESULT")
print("=" * 60)
print(result.redacted_text)
print("\n" + "=" * 60)
print(f"PII Items Redacted: {result.redaction_count}")
print(f"PII Types Found: {result.pii_types_found}")
print(f"Processing Time: {result.processing_time_ms:.2f}ms")

In [ ]:
# Verify financial data preserved
assert "$4,250.00" in result.redacted_text, "Financial data should be preserved!"
assert "$85,000.00" in result.redacted_text, "YTD should be preserved!"
assert "123-45-6789" not in result.redacted_text, "SSN should be redacted!"
print("✅ Verification passed: Financial data preserved, PII redacted!")

---
## 7️⃣ Complete Example: High-Income Married Couple

In [ ]:
def full_analysis(profile: UserFinancialProfile):
    """Run complete tax analysis."""
    
    print("🛡️ TAXGUARD AI - COMPLETE ANALYSIS")
    print("=" * 70)
    
    calc = TaxCalculator()
    result = calc.calculate_tax(profile)
    
    print(f"\n📊 TAX SUMMARY")
    print(f"   Filing: {profile.filing_status.value.replace('_', ' ').title()}")
    print(f"   Gross Income: ${result.gross_income:,.2f}")
    print(f"   Taxable Income: ${result.taxable_income:,.2f}")
    print(f"   Total Tax: ${result.total_tax_liability:,.2f}")
    print(f"   Effective Rate: {result.effective_rate:.1f}%")
    print(f"   Marginal Rate: {result.marginal_rate*100:.0f}%")
    
    if result.refund_or_owed >= 0:
        print(f"\n   ✅ PROJECTED REFUND: ${result.refund_or_owed:,.2f}")
    else:
        print(f"\n   ⚠️ PROJECTED OWED: ${abs(result.refund_or_owed):,.2f}")
    
    # Recommendations
    eng = RecommendationEngine()
    recs = eng.generate_recommendations(profile)
    
    print(f"\n💡 TOP RECOMMENDATIONS")
    for rec in recs.basic_recommendations[:3]:
        print(f"   • {rec.title}: Save ${rec.potential_tax_savings:,.2f}")
    
    print(f"\n🚀 ADVANCED OPPORTUNITIES ({len(recs.advanced_recommendations)})")
    for rec in recs.advanced_recommendations[:3]:
        print(f"   • {rec.title}")
    
    print(f"\n💰 MAX POTENTIAL SAVINGS: ${recs.max_potential_savings:,.2f}")
    
    return result, recs

# Run analysis on our profile
result, recs = full_analysis(std_profile)

---
## 📊 Summary

TaxGuard AI provides:

| Feature | Description |
|---------|-------------|
| **Multiple Income Sources** | Spouse, multiple jobs, 1099s, rental |
| **YTD Tracking** | Accurate refund/owed projections |
| **Basic Recommendations** | 401(k), HSA, IRA optimization |
| **Advanced Strategies** | Business formation, Section 179, real estate |
| **What-If Simulations** | Test impact of changes |
| **PII Redaction** | Privacy air gap for AI safety |

### Key Security Features:
- ✅ PII is **NEVER** sent to AI
- ✅ Tax calculations happen in **Python**, not LLM
- ✅ Tax brackets are **hardcoded** (no AI hallucination)